In [3]:
from os import listdir
from os.path import isfile, join

mypath = "./about_father"

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
import pandas as pd
import numpy as np

In [87]:
df1 = pd.read_csv(mypath+"/"+onlyfiles[1])

In [88]:
df1.describe()

,F#,Score,nose_score,nose_x,nose_y,leftEye_score,leftEye_x,leftEye_y,rightEye_score,rightEye_x,...,leftKnee_y,rightKnee_score,rightKnee_x,rightKnee_y,leftAnkle_score,leftAnkle_x,leftAnkle_y,rightAnkle_score,rightAnkle_x,rightAnkle_y
count,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,...,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000
mean,59.000000,0.730161,0.992031,126.527866,92.325175,0.988604,133.169384,85.849701,0.980366,116.939470,...,348.828742,0.119965,62.529870,328.395564,0.028272,156.972191,312.402723,0.033796,9.963195,305.428189
std,34.496377,0.007808,0.001444,2.104167,1.085571,0.001950,2.185965,1.050918,0.003138,2.331868,...,5.436410,0.016825,51.989397,15.302903,0.005368,58.459163,6.792290,0.007986,16.703449,13.891771
min,0.000000,0.708791,0.986453,120.689889,90.016569,0.981223,128.072661,83.135230,0.969309,112.071198,...,328.710646,0.089011,-2.644092,306.701757,0.019313,-3.202479,301.988979,0.020396,-10.138629,168.314452
25%,29.500000,0.725658,0.991679,125.942279,91.584814,0.987883,132.261412,85.093973,0.978917,115.022846,...,349.034883,0.107639,0.523590,311.323080,0.024875,161.993544,308.836767,0.027973,-2.169508,304.125804
50%,59.000000,0.732342,0.992320,126.960681,92.547532,0.988855,133.248778,85.797324,0.981300,116.065331,...,350.537563,0.118205,104.208625,337.106372,0.026763,177.993754,310.299861,0.032441,13.436236,306.388631
75%,88.500000,0.736587,0.992857,127.828261,93.141316,0.989773,134.668519,86.676628,0.982451,118.607070,...,351.294275,0.130448,106.088919,339.871718,0.030939,180.077918,311.513332,0.038932,17.247942,307.930363
max,118.000000,0.742096,0.994154,130.548622,94.708865,0.992078,136.867038,88.412864,0.986319,123.906681,...,353.412773,0.176178,126.447953,356.396679,0.046812,204.266844,328.207835,0.056101,94.042275,321.465600


In [94]:
len(df1.columns.tolist()),424/53

(53, 8.0)

In [16]:
df1.describe().values.flatten().shape

(424,)

In [17]:
# Read all the values, if file contains father then 1 else label is 0
# Binary Classification

In [18]:
X_fathers = []
X_about = []
for filename in onlyfiles:
    df1 = pd.read_csv(mypath+"/"+filename)
    x = df1.describe().values.flatten()
    if "father" in filename.lower():
        X_fathers.append(x)
    else:
        X_about.append(x)

print(len(X_fathers),len(X_about))

194 236


In [21]:
from sklearn.model_selection import train_test_split

y_fathers= [1]*len(X_fathers)
y_about = [0]*len(X_about)


In [ ]:
X_fathers.extend(X_about)
y_fathers.extend(y_about)

In [31]:
X = np.array(X_fathers)
y = np.array(y_fathers)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle =True)

In [37]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape,sum(y_train)/288,sum(y_test)/142

((288, 424),
 (142, 424),
 (288,),
 (142,),
 0.4201388888888889,
 0.5140845070422535)

In [45]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [39]:
clf = SVC(gamma='auto')

In [40]:
clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
from sklearn.metrics import accuracy_score
preds_val = clf.predict(X_test)
accuracy = accuracy_score(y_test, preds_val)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 48.59%


In [55]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    kernels = ["linear","poly", "rbf", "sigmoid"]
    param_grid = {'C': Cs, 'gamma' : gammas, 'kernel' : kernels}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds, n_jobs=-1, verbose=2)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [56]:
svc_param_selection(X_train,y_train,5)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    3.4s finished


{'C': 0.001, 'gamma': 0.001, 'kernel': 'poly'}

In [57]:
clf = SVC(gamma=0.001, C=0.001, kernel = 'poly')

In [58]:
clf.fit(X_train,y_train)

SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [80]:
preds_val = clf.predict(X_test)
accuracy = accuracy_score(y_test, preds_val)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 95.07%


In [60]:
from sklearn.tree import DecisionTreeClassifier

dclf = DecisionTreeClassifier(random_state=42)

In [62]:
dclf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [63]:
preds_val = dclf.predict(X_test)
accuracy = accuracy_score(y_test, preds_val)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.66%


In [77]:
def dtree_param_selection(X, y, nfolds):
    max_features = [1, 10, 50, 100, 300]
    max_depth =   np.arange(1, 21)
    num_leafs = [1, 5, 10, 20, 50, 100]
    param_grid = {'max_features': max_features, 'max_depth' : max_depth, 'min_samples_leaf':num_leafs}
    grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=nfolds, n_jobs=-1, verbose=2)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [78]:
dtree_param_selection(X_train,y_train,5)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:    3.0s finished
c:\users\praty\miniconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': 3, 'max_features': 300, 'min_samples_leaf': 5}

In [81]:
dclf = DecisionTreeClassifier(random_state=42,max_depth=3,max_features=300, min_samples_leaf=5)
dclf.fit(X_train,y_train)
preds_val = dclf.predict(X_test)
accuracy = accuracy_score(y_test, preds_val)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 95.07%


In [82]:
from sklearn_porter import Porter

In [85]:
porter = Porter(clf, language='java')
output = porter.export(embed_data=True)
print(output)
with open('SVCClassifier.java', 'w') as f:
         f.write(output)

class SVC {

    private enum Kernel { LINEAR, POLY, RBF, SIGMOID }

    private int nClasses;
    private int nRows;
    private int[] classes;
    private double[][] vectors;
    private double[][] coefficients;
    private double[] intercepts;
    private int[] weights;
    private Kernel kernel;
    private double gamma;
    private double coef0;
    private double degree;

    public SVC (int nClasses, int nRows, double[][] vectors, double[][] coefficients, double[] intercepts, int[] weights, String kernel, double gamma, double coef0, double degree) {
        this.nClasses = nClasses;
        this.classes = new int[nClasses];
        for (int i = 0; i < nClasses; i++) {
            this.classes[i] = i;
        }
        this.nRows = nRows;

        this.vectors = vectors;
        this.coefficients = coefficients;
        this.intercepts = intercepts;
        this.weights = weights;

        this.kernel = Kernel.valueOf(kernel.toUpperCase());
        this.gamma = gamma;
        thi

In [86]:
porter = Porter(dclf, language='java')
output = porter.export(embed_data=True)
print(output)
with open('DecisionTreeClassifier.java', 'w') as f:
         f.write(output)

class DecisionTreeClassifier {

    private static int findMax(int[] nums) {
        int index = 0;
        for (int i = 0; i < nums.length; i++) {
            index = nums[i] > nums[index] ? i : index;
        }
        return index;
    }

    public static int predict(double[] features) {
        int[] classes = new int[2];
            
        if (features[246] <= 163.59510803222656) {
            if (features[403] <= 0.9494742751121521) {
                if (features[122] <= 3.516611337661743) {
                    classes[0] = 0; 
                    classes[1] = 95; 
                } else {
                    classes[0] = 2; 
                    classes[1] = 3; 
                }
            } else {
                if (features[405] <= 261.5179748535156) {
                    classes[0] = 13; 
                    classes[1] = 0; 
                } else {
                    classes[0] = 1; 
                    classes[1] = 4; 
                }
            }
        } else {
